# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [8]:
# your code here
orders=pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [11]:
orders.columns=orders.columns.str.lower().str.replace(" ","_")

In [22]:
orders.shape

(397924, 14)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.


#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

In [28]:
spent_by_client=orders.groupby("invoiceno")["amount_spent"].sum()

In [29]:
spent_by_client

invoiceno
536365    139.12
536366     22.20
536367    278.73
536368     70.05
536369     17.85
           ...  
581583    124.60
581584    140.64
581585    329.05
581586    339.20
581587    249.45
Name: amount_spent, Length: 18536, dtype: float64

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

In [50]:
percentil_95=spent_by_client.quantile(0.95)
percentil_95

1236.5425

In [49]:
percentil_75=spent_by_client.quantile(0.75)
percentil_75

471.66

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

In [55]:
orders["VIP"]=orders["invoiceno"].apply(
    lambda x:"VIP" if spent_by_client[x]>percentil_95 else 
    "Preferred" if spent_by_client[x]>percentil_75 else 
    "Regular")

In [57]:
orders["VIP"].value_counts()

VIP
Regular      235084
Preferred    121507
VIP           41333
Name: count, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [67]:
# your code here
orders.groupby(["country","VIP"]).size().unstack(fill_value=0).sort_values(by="VIP",ascending=False).head(1)

VIP,Preferred,Regular,VIP
country,,,
United Kingdom,103027,222700,28618


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [96]:
# your code here

# Nos quedamos solo con los clientes VIP y Preferred
orders_pref_vip=orders[orders["VIP"].isin(["VIP","Preferred"])]
orders_pref_vip.head(5)

,unnamed:_0,invoiceno,stockcode,year,month,day,hour,description,quantity,invoicedate,unitprice,customerid,country,amount_spent,VIP
26,26,536370,22728,2010,12,3,8,alarm clock bakelike pink,24,2010-12-01 08:45:00,3.75,12583,France,90.0,Preferred
27,27,536370,22727,2010,12,3,8,alarm clock bakelike red,24,2010-12-01 08:45:00,3.75,12583,France,90.0,Preferred
28,28,536370,22726,2010,12,3,8,alarm clock bakelike green,12,2010-12-01 08:45:00,3.75,12583,France,45.0,Preferred
29,29,536370,21724,2010,12,3,8,panda and bunnies sticker sheet,12,2010-12-01 08:45:00,0.85,12583,France,10.2,Preferred
30,30,536370,21883,2010,12,3,8,stars gift tape,24,2010-12-01 08:45:00,0.65,12583,France,15.6,Preferred


In [93]:
# Hacemos groupby para contar clientes preferred y VIP por país
orders_pref_vip_counts=orders_pref_vip.groupby("country")["VIP"].count()
orders_pref_vip_counts.head(5)

country
Australia     910
Austria       333
Belgium      1099
Brazil         32
Canada        136
Name: VIP, dtype: int64

In [97]:
# Lo ordenamos de mayor a menor
orders_pref_vip_counts.sort_values(ascending=False).head(1)

country
United Kingdom    131645
Name: VIP, dtype: int64